In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras

from keras import callbacks,layers, Sequential

import seaborn as sns


In [9]:
neuralNetModel = Sequential([
    layers.InputLayer(input_shape=(100,)),
    
    layers.BatchNormalization(renorm=True),
    layers.Dense(128, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.BatchNormalization(renorm=True),
    layers.Dense(64, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.BatchNormalization(renorm=True),
    layers.Dense(32, activation='relu'),
    layers.Dropout(rate = 0.3),
    layers.Dense(1, activation='sigmoid'),
])

print(neuralNetModel.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_12 (Bat  (None, 100)              700       
 chNormalization)                                                
                                                                 
 dense_16 (Dense)            (None, 128)               12928     
                                                                 
 dropout_12 (Dropout)        (None, 128)               0         
                                                                 
 batch_normalization_13 (Bat  (None, 128)              896       
 chNormalization)                                                
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 dropout_13 (Dropout)        (None, 64)               

In [ ]:
def NeuralNetwork_test(path, epochs=50, test_size=0.3):
    # Import dataset
    df_dataset = pd.read_csv(path)

    # Encode Labels for numeric classification
    label_encoder = LabelEncoder()
    df_dataset['Label'] = label_encoder.fit_transform(df_dataset['Label'])

    # Set a random state for sampling
    RANDOM_STATE_SEED = np.random.randint(123)
    print(f'Random Seed:{RANDOM_STATE_SEED}')
    
    # Split dataset in train and test
    train, test = train_test_split(df_dataset, test_size=test_size, random_state=RANDOM_STATE_SEED)
    
    # Count how many instances there are in each label
    print(df_dataset["Label"].value_counts())

    # Separate in X and y for better classification
    y_train = np.array(train.pop("Label"))# pop removes "Label" from the dataframe
    X_train = train.values

    print(f'Tipo X_train: {type(X_train)} Tipo y_train: {type(y_train)} Shape X_train:{X_train.shape} Shape y_train: {y_train.shape}')

    y_test = np.array(test.pop("Label")) # pop removes "Label" from the dataframe
    X_test = test.values

    print(f'Tipo X_test: {type(X_test)} Tipo y_test: {type(y_test)} Shape X_test:{X_test.shape} Shape y_test: {y_test.shape}')
    
    # Define the used classification model
    neuralNetModel = keras.Sequential([
        layers.InputLayer(input_shape=(X_train.shape[1],)),
        
        layers.BatchNormalization(renorm=True),
        layers.Dense(128, activation='relu'),
        layers.Dropout(rate = 0.3),
        layers.BatchNormalization(renorm=True),
        layers.Dense(64, activation='relu'),
        layers.Dropout(rate = 0.3),
        layers.BatchNormalization(renorm=True),
        layers.Dense(32, activation='relu'),
        layers.Dropout(rate = 0.3),
        layers.Dense(1, activation='sigmoid'),
    ])

    print(neuralNetModel.summary())

    # %%
    
    optimizer = tf.keras.optimizers.Adam(epsilon=0.01)


    early_stopping = callbacks.EarlyStopping(
        min_delta = 0.001,
        patience = 5,
        restore_best_weights = True
    )


    neuralNetModel.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['binary_accuracy'],
    )

    history = neuralNetModel.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size = 256,
        callbacks=[early_stopping]
    )

    history_frame = pd.DataFrame(history.history)
    history_frame.loc[:, ['loss']].plot()
    history_frame.loc[:, ['binary_accuracy']].plot()

    
    predictions = (neuralNetModel.predict(X_test) > 0.5).astype("int32")
    print(classification_report(y_test,predictions,digits=10))
    print(f'Accuracy: {accuracy_score(y_test, predictions)}')

    # %%
    cf_matrix = confusion_matrix(y_test, predictions)
    
    sns.heatmap(cf_matrix, annot=True)   

In [ ]:
NeuralNetwork_test('/home/mintssj/Downloads/datasets/filtered/NSL_KDD_equal.csv',epochs=10)